In [1]:
#Scrapping data from OJK
#Install library first
%pip install requests
%pip install beautifulsoup4
%pip install pypdf2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\GABRIEL VICERAIRA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\GABRIEL VICERAIRA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: C:\Users\GABRIEL VICERAIRA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
#Import library
import requests
from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfFileReader
import numpy as np


In [18]:
# Fungsi untuk mendapatkan semua link ke halaman laporan tahunan
def get_report_links(url, keyword, start_year, end_year):
    report_links = []
    page_number = 1
    while True:
        response = requests.get(url, params={'page': page_number})
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Cari semua tag <a> yang mengandung link ke laporan
        found_links = False
        for a_tag in soup.find_all('a', href=True):
            link_text = a_tag.get_text().strip()
            for year in range(start_year, end_year + 1):
                if keyword in link_text and str(year) in link_text:
                    main_address = "https://ojk.go.id/"
                    report_links.append(main_address+a_tag['href'])
                    found_links = True
        
        # Cek jika ada nomor halaman berikutnya
        pagination_links = soup.find_all('a', class_='bluebutton')
        next_page_exists = any(a_tag.get_text().isdigit() and int(a_tag.get_text()) == page_number + 1 for a_tag in pagination_links)
        
        if not next_page_exists or not found_links:
            break
        
        
        page_number += 1
    
    return report_links

# Fungsi untuk mendapatkan link PDF dari halaman laporan tahunan
def get_pdf_link(report_url):
    pdf_links = []
    response = requests.get(report_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for a_tag in soup.find_all('a', href=True):
        if a_tag['href'].endswith('.pdf'):
            pdf_links.append(a_tag['href'])
    
    return pdf_links

In [16]:
# Masukkan kata kunci dan rentang tahun
url_main = "https://ojk.go.id/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/default.aspx"
keyword = "Statistik Perbankan Indonesia"
start_year = 2003
end_year = 2023

# Ambil semua link ke laporan bulanan dari tahun ke tahun
report_links = get_report_links(url_main, keyword, start_year, end_year)

# Untuk setiap link laporan bulanan, ambil link PDF
pdf_links = []
for report_link in report_links:
    pdf_links.extend(get_pdf_link(report_link))

# Tampilkan semua link PDF yang ditemukan
for pdf_link in pdf_links:
    print(pdf_link)

/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/Documents/Pages/Statistik-Perbankan-Indonesia---Desember-2023/STATISTIK%20PERBANKAN%20INDONESIA%20-%20DESEMBER%202023.pdf
/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/Documents/Pages/Statistik-Perbankan-Indonesia---November-2023/STATISTIK%20PERBANKAN%20INDONESIA%20-%20NOVEMBER%202023.pdf
/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/Documents/Pages/Statistik-Perbankan-Indonesia---Oktober-2023/STATISTIK%20PERBANKAN%20INDONESIA%20-%20OKTOBER%202023.pdf
/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/Documents/Pages/Statistik-Perbankan-Indonesia---September-2023/STATISTIK%20PERBANKAN%20INDONESIA%20-%20SEPTEMBER%202023.pdf


In [17]:
#save ke csv
import csv

def save_to_csv(pdf_links, filename='pdf_links.csv'):
    # Buka atau buat file CSV
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Tulis header (opsional)
        writer.writerow(['PDF Link'])
        
        # Tulis setiap link PDF ke baris baru
        for link in pdf_links:
            writer.writerow([link])
    
    print(f'All links saved to {filename}')

save_to_csv(pdf_links)

All links saved to pdf_links.csv


In [ ]:
#unduh pdf dan baca halaman tertentu dari pdf